In [ ]:
#!pip install numpy torch torchvision matplotlib scipy

# PyTorch Device

In [ ]:
import torch

In [ ]:
# Choose torch device

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print("Using device:", device)

# Dataset Utils

In [ ]:
import dataset
from dataset import *

# MI Utils

In [ ]:
import mi
from mi import *    

# Neural Network Utils

In [ ]:
import nn
from nn import *

# Experiments

In [ ]:
import importlib
importlib.reload(dataset)
importlib.reload(mi)
importlib.reload(nn)

In [ ]:
# dataset = buildDatasets( *loadMNISTData(root="data"), ratio=6/7, name="mnist" )
dataset = buildDatasets( *loadSyntheticData(file="data/synthetic/var_u.mat"), name="synthetic" )

In [ ]:
print( "Number of samples in the training set:", len(dataset["train"]) )
print( "Number of samples in the test set:", len(dataset["test"]) )

In [ ]:
# SETTING 1
# - Synthetic dataset
# - Test accuracy: 0.9426

setup = dict()
# network parameters
setup["hidden_dims"] = [10, 7, 5, 4, 3]                         
setup["output_dim"] = 1                                         
setup["hidden_activation_f"] = lambda input: F.tanh(input)      
setup["output_activation_f"] = lambda input: F.sigmoid(input)   

# optimizer
setup["lr"] = 0.01                             
setup["momentum"] = 0.9
setup["optimizer"] = lambda parameters: torch.optim.SGD( parameters, lr=setup["lr"], momentum=setup["momentum"] )

# training configuration
setup["n_epochs"] = 10000
setup["batch_size"] = None
setup["loss_function"] = lambda output, target, reduction='mean': F.binary_cross_entropy(output.reshape(-1), target.float(), reduction=reduction)
setup["evaluate_correct"] = lambda output, target: torch.sum( torch.round(output.reshape(-1)) == target, dtype=torch.float32 )


In [ ]:
# SETTING 2
# - MNIST
# - Test accuracy: 0.9027

setup = dict()
# network parameters
setup["hidden_dims"] = [256, 256, 128]                                          # hidden layers sizes
setup["output_dim"] = 10                                                        # output layer size
setup["hidden_activation_f"] = lambda input: F.tanh(input)                      # activation function for the hidden layers
setup["output_activation_f"] = lambda input: F.log_softmax( input, dim=1 )      # activation function for the output layer

# optimizer
setup["lr"] = 0.01  
setup["optimizer"] = lambda parameters: torch.optim.Adam( parameters, lr=setup["lr"] )       

# training configuration
setup["n_epochs"] = 20
setup["batch_size"] = None
setup["loss_function"] = lambda output, target, reduction='mean': F.nll_loss(output, target, reduction=reduction)
setup["evaluate_correct"] = lambda output, target: torch.sum( output.argmax(dim=1) == target, dtype=torch.float32 )

In [ ]:
loader = buildDataLoader(dataset, batch_size=setup["batch_size"])
# batch = next( iter( loader["train"] ) )

In [ ]:
save = True
save_interval = 1

model = Network(
            input_dim=dataset["n_features"], 
            hidden_dims=setup["hidden_dims"],
            output_dim=setup["output_dim"],
            hidden_activation_f=setup["hidden_activation_f"],
            output_activation_f=setup["output_activation_f"]
        ).to(device)

optimizer = setup["optimizer"]( model.parameters() )

if save:
        folder = save_setup(dataset, setup)

for epoch in range(1, setup["n_epochs"] + 1):
        train(model, setup, loader["train"], optimizer, device, epoch, verbose=2)
        test(model, setup, loader["test"], device)
        if save and epoch%save_interval == 0:
                save_activations(model, dataset["full"], epoch, folder, device)

In [ ]:
mi_xt_epochs, mi_ty_epochs, epochs = compute_mi(dataset["full"], folder, interval=1)

In [ ]:
# Save MI results as compressed numpy file
path = "./save/"
np.savez_compressed( path+folder+"/mi", mi_xt_epochs=mi_xt_epochs, mi_ty_epochs=mi_ty_epochs, epochs=epochs)

In [ ]:
# Load results
loaded_results = np.load( path+folder+"/mi.npz" )
loaded_mi_xt_epochs = loaded_results["mi_xt_epochs"]
loaded_mi_ty_epochs = loaded_results["mi_ty_epochs"]
loaded_epochs = loaded_results["epochs"]

In [ ]:
plot_info_plan(mi_xt_epochs, mi_ty_epochs, epochs)